# Test wind forecast with barometric pressure

In [1]:
# install the modules on the OS
#!pip install influxdb
#!pip3 install ipynb

# import the modules
import pandas as pd
import datetime as dt
import numpy as np
import influxdb
import os
import math
import scipy.stats as st
import plotly.graph_objects as go

from ipynb.fs.full.config_and_functions import *

In [29]:
def get_data_in_current_time_period2(data, days_back, days_forward):
    
    last_entry = data.iloc[-1:]
    datetime_of_last_entry = get_datetime_of_last_entry()
    
    print('datetime_of_last_entry: {}'.format(datetime_of_last_entry))
    
    #month_back = int((last_entry.index[0] + dt.timedelta(days=-days_back)).strftime('%-m'))
    #month_forward = int((last_entry.index[0] + dt.timedelta(days=days_forward)).strftime('%-m'))
    #day_back = int((last_entry.index[0] + dt.timedelta(days=-days_back)).strftime('%-d'))
    #day_forward = int((last_entry.index[0] + dt.timedelta(days=days_forward)).strftime('%-d'))
    
    month_back = int((datetime_of_last_entry[0] + dt.timedelta(days=-days_back)).strftime('%-m'))
    month_forward = int((datetime_of_last_entry[0] + dt.timedelta(days=days_forward)).strftime('%-m'))
    day_back = int((datetime_of_last_entry[0] + dt.timedelta(days=-days_back)).strftime('%-d'))
    day_forward = int((datetime_of_last_entry[0] + dt.timedelta(days=days_forward)).strftime('%-d'))
    print('month_back: {}'.format(month_back))
    print('day_back: {}'.format(day_back))
    print('month_forward: {}'.format(month_forward))
    print('day_forward: {}'.format(day_forward))
    
    print('len of data: {}'.format(len(data)))

    df_filtered = data[
        ((data.index.month == month_back) & (data.index.day == day_back)
        |
        (data.index.month == month_forward) & (data.index.day == day_forward))
    ]
    
    print('len of df_filtered: {}'.format(len(df_filtered)))

    print('df_filtered')
    print(df_filtered)

    return df_filtered




def get_air_temperature_forecast_values():
    
    # get datetime of last entry
    datetime_of_last_entry = get_datetime_of_last_entry()
    
    # Get data for air temperature prediction
    query = """SELECT
            last(air_temperature) AS last_air_temperature,
            last(dew_point) AS last_dew_point,
            last(humidity) AS last_humidity,
            last(barometric_pressure_qfe) AS last_barometric_pressure_qfe
            FROM "meteorology"."autogen"."{}" """.format(station_name)

    df_last = pd.DataFrame(client.query(query)[station_name])
    
    last_air_temperature = float(df_last["last_air_temperature"].values)
    last_dew_point = float(df_last["last_dew_point"].values)
    last_humidity = float(df_last["last_humidity"].values)
    last_barometric_pressure_qfe = float(df_last["last_barometric_pressure_qfe"].values)

    #print(last_air_temperature)
    #print(last_dew_point)
    #print(last_humidity)
    #print(last_barometric_pressure_qfe)

    # find similar conditions, stop if something found
    # "simulate" HAVING clause
    len_of_df = 0
    factor = 0.05
    while len_of_df <= 20:
        
        query = """SELECT
                (air_temperature),
                (dew_point),
                (humidity),
                (barometric_pressure_qfe)
                FROM "meteorology"."autogen"."{}"
                WHERE
                    (air_temperature > {} AND air_temperature < {})
                    AND
                    (dew_point > {} AND dew_point < {})
                    AND
                    (humidity > {} AND humidity < {})
                    AND
                    (barometric_pressure_qfe > {} AND barometric_pressure_qfe < {})
                    AND
                    (time < now() - 3d)""".format(
                            station_name,
                            last_air_temperature - factor * 1,
                            last_air_temperature + factor * 1,
                            last_dew_point - factor * 20,
                            last_dew_point + factor * 20,
                            last_humidity - factor * 20,
                            last_humidity + factor * 20,
                            last_barometric_pressure_qfe - factor * 40,
                            last_barometric_pressure_qfe + factor * 40)

        df = pd.DataFrame(client.query(query)[station_name])
        len_of_df = len(df)
        factor += 0.1
        if factor == 10:
            break
    

    #print('df:\n', df, '\n---', 'len(df)', len(df))
    

    # loop in widening day span, break if span is 60 (2 x 30) days
    days_span = 0
    len_of_df_f = 0
    
    #print('filter for current time period')
    
    #get_data_in_current_time_period2(df, 1, 1)

   
    while len_of_df_f <= 5:
        
        df_f = df.copy()
        df_f = get_data_in_current_time_period2(df_f, days_span + 1, days_span + 1) # Filter for current time period
        if days_span == 30:
            break
            
        len_of_df_f = len(df_f)
        days_span += 1

        
    #print('len_of_df_f: {}'.format(len_of_df_f))
    #print('df_f:\n', df_f, '\n---')
    
    
    # loop in widening timespan, break if dataframe has length of 1
    minutes_span = 0
    len_of_df_ff = 0
    while len_of_df_ff <= 0:

        df_ff = df_f.copy()
        df_ff = df_ff.between_time(
            ((datetime_of_last_entry + dt.timedelta(minutes=-minutes_span)).strftime('%H:%M'))[0],
            ((datetime_of_last_entry).strftime('%H:%M'))[0]
        )
        if len(df_ff) == 1: break

        df_ff = df_ff.between_time(
            ((datetime_of_last_entry).strftime('%H:%M'))[0],
            ((datetime_of_last_entry + dt.timedelta(minutes=minutes_span)).strftime('%H:%M'))[0]
        )
        if len(df_ff) == 1: break

        len_of_df_ff = len(df_ff)
        minutes_span += 10

    #print('minutes_span: {}'.format(minutes_span))
    #print('len_of_df_ff: {}'.format(len_of_df_ff))
    #print('df_ff:\n', df_ff, '\n---')


    datetime_start = (df_ff.iloc[0:].index[0]
                          .replace(
                              hour=datetime_of_last_entry.hour[0],
                              minute=datetime_of_last_entry.minute[0]
                          )
                     ) 
    datetime_end = (df_ff.iloc[0:].index[0] + dt.timedelta(days=3))
    

    print('datetime_start: {}'.format(datetime_start))
    print('datetime_end: {}'.format(datetime_end))
    
    query = """SELECT
                (air_temperature)
                FROM "meteorology"."autogen"."{}"
                WHERE
                    (time >= '{}' AND time <= '{}')""".format(
                        station_name,
                        datetime_start.tz_localize(tz = None),
                        datetime_end.tz_localize(tz = None)
    )
    
    df_final = pd.DataFrame(client.query(query)[station_name])
    
    print('df_final')
    print(df_final)
    
    #starting_datetime = df_ff.iloc[0:].index[0]
    #print('starting_datetime: {}'.format(starting_datetime))

    forecast_values = []
    forecast_values.append(df_final.loc[datetime_start + dt.timedelta(hours=2)].values)
    forecast_values.append(df_final.loc[datetime_start + dt.timedelta(hours=4)].values)
    forecast_values.append(df_final.loc[datetime_start + dt.timedelta(hours=8)].values)
    forecast_values.append(df_final.loc[(datetime_start + dt.timedelta(hours=24)).replace(hour=12, minute=0)].values)
    forecast_values.append(df_final.loc[(datetime_start + dt.timedelta(hours=48)).replace(hour=12, minute=0)].values)


    print('2h: {}'.format(forecast_values[0]))
    print(df_final.loc[datetime_start + dt.timedelta(hours=2)])
    print('4h: {}'.format(forecast_values[1]))
    print(df_final.loc[datetime_start + dt.timedelta(hours=4)])
    print('8h: {}'.format(forecast_values[2]))
    print(df_final.loc[datetime_start + dt.timedelta(hours=8)])
    print('morgen mittag: {}'.format(forecast_values[3]))
    print(df_final.loc[(datetime_start + dt.timedelta(hours=24)).replace(hour=12, minute=0)])
    print('übermorgen mittag: {}'.format(forecast_values[4]))
    print(df_final.loc[(datetime_start + dt.timedelta(hours=48)).replace(hour=12, minute=0)])
    
    #print(df_final.loc[starting_datetime + dt.timedelta(hours=4)])
    
    return forecast_values

get_air_temperature_forecast_values()

datetime_of_last_entry: DatetimeIndex(['2020-01-06 16:10:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
month_back: 1
day_back: 5
month_forward: 1
day_forward: 7
len of data: 635
len of df_filtered: 2
df_filtered
                           air_temperature  dew_point  humidity  \
2012-01-07 13:00:00+00:00              2.6        0.1        85   
2012-01-07 13:10:00+00:00              2.7        0.4        85   

                           barometric_pressure_qfe  
2012-01-07 13:00:00+00:00                    975.3  
2012-01-07 13:10:00+00:00                    975.3  
datetime_of_last_entry: DatetimeIndex(['2020-01-06 16:10:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
month_back: 1
day_back: 4
month_forward: 1
day_forward: 8
len of data: 635
len of df_filtered: 26
df_filtered
                           air_temperature  dew_point  humidity  \
2008-01-08 03:30:00+00:00              2.5        0.3        85   
2008-01-08 04:00:00+00:00              2.5        0.5        86   
2

[array([2.9]), array([1.5]), array([0.3]), array([0.5]), array([3.5])]